In [0]:
from Helpers import *

sc = SparkContext.getOrCreate()

## Input Parameters

In [0]:
# Get Input Parameters

print(f"AWS_REGION: {AWS_REGION}")
print(f"LOAD_DATE: {LOAD_DATE}")
print(f"STATUS_TABLE: {STATUS_TABLE}")
print(f"SOURCE_BUCKETS: {SOURCE_BUCKETS}")
print(f"RWE_BUCKET: {RWE_BUCKET}")
print(f"SNS_TOPIC: {SNS_TOPIC}")
print(f"PROCESSING_JOB: {PROCESSING_JOB}")
print(f"PROCESSING_TIMEOUT: {PROCESSING_TIMEOUT}")
print(f"MAX_JOB_THREADS: {MAX_JOB_THREADS}")

## Submit Jobs

In [0]:
# Create Status Handler Object
process_status_handler = SuperiorProcessStatusHandler(AWS_REGION, STATUS_TABLE, SNS_TOPIC, LOAD_DATE)

# Get Pending Batches to be Processed (consider only pqt files)
pending_batches = process_status_handler.get_items_with_status('Pending', LOAD_DATE)
pending_batches = [batch for batch in pending_batches if batch['Format'] == 'pqt']
for batch in pending_batches:
    print(f"Pending batch: {batch}")

In [0]:
futures = []
params = dbutils.widgets.getAll()
with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_JOB_THREADS) as executor:
    # Data-Processor
    for i,batch in enumerate(pending_batches):
        params = dbutils.widgets.getAll()
        params['batch'] = batch['Key']
        futures.append(executor.submit(dbutils.notebook.run, PROCESSING_JOB, PROCESSING_TIMEOUT, params))
    
    concurrent.futures.wait(futures)
    results = [future.result() for future in futures]
    print(f"Completed {len(results)} batches")
    for res in results:
        print("Completed batch: ", res)